In [1]:
# 읽어올 파일 이름
file_name_train = "data/train_data.csv"
file_name_test  = "data/test_data.csv"


In [3]:
import pandas as pd
# 파일에 데이터가 탭으로 구분되어있음
train_data = pd.read_csv(file_name_train, encoding='utf-8-sig')
test_data = pd.read_csv(file_name_test, encoding='utf-8-sig')

In [68]:
print(train_data.head())
print(test_data.head())

                                      review_content  rating  label
0  숙박 예약했는데 분정도 일찍갔더니 대실비 내고 짐가져다놓으라길래 그냥 할일 먼저하고...       0      0
1                          자주가는 숙소입니다 이번에는 청소가 별로였어요       1      0
2    금연방인데 담배냄새인지 비슷한냄새가 계속 나서 숨쉬기 불편했어요그래도 잘 쉬었다가가요       3      0
3                                    대실 시간 길어서 너무좋아요       5      1
4                                       좋아요좋아요좋아요좋아요       5      1
                                      review_content  rating  label
0  부모님 모시고 처음으로 여행을 가봤는데 일단 룸 형식은 너무 마음에 들었습니다 체크...       1      0
1                                     역시나깔끔하게이용잘했습니다       5      1
2                                       화장실 천장이 뚫려있음       3      0
3                                     숙소 엄청 깔끔하고 좋아요       5      1
4  에어컨이 개더워요 바람도 약하고 걍 개 더워요 아무데서나 잘 자는편인데 잠을 못잤네...       3      0


In [4]:
# 훈련용 리뷰 데이터 개수 출력
print(f"훈련용 리뷰 개수 : {len(train_data)}")
# 테스트용 리뷰 데이터 개수 출력
print(f"테스트용 리뷰 개수 : {len(test_data)}")

훈련용 리뷰 개수 : 20609
테스트용 리뷰 개수 : 5153


In [5]:
# 중복 제거
train_data.drop_duplicates(subset=['text'], inplace=True)
test_data.drop_duplicates(subset=['text'], inplace=True)
# y_review.drop_duplicates(subset=['review_content'], inplace=True)

In [6]:
import numpy as np
# 결측치 제거
train_data['text'] = train_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
train_data['text'] = train_data['text'].str.replace('^ +','',regex=True)
train_data.dropna(how='any', inplace=True)

In [7]:
test_data['text'] = test_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
test_data['text'] = test_data['text'].str.replace('^ +','',regex=True)
test_data.dropna(how='any', inplace=True)

In [8]:
# 중복, 결측치 제거 후 데이터 개수
print(f"전처리 후 훈련 데이터 개수 : {train_data.shape}")
print(f"전처리 후 테스트 데이터 개수 : {test_data.shape}")
# print(f"전처리 후 테스트 데이터 개수 : {y_review.shape}")

전처리 후 훈련 데이터 개수 : (20609, 5)
전처리 후 테스트 데이터 개수 : (5153, 5)


In [9]:
# 불용어 목록 선언
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다', '용하다', '물이',
    '뿐', '대로', '만', '따름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [10]:
# 형태소 분석기 객체 생성
from konlpy.tag import Okt, Kkma, Komoran, Hannanum
okt  = Okt()

In [11]:
# 작업의 진행 상황을 시각적으로 표시해주는 라이브러리
from tqdm import tqdm

In [13]:
X_train = []
y_train = []

for sentence, label in tqdm(zip(train_data['text'], train_data['label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_train.append(filtered)
        y_train.append(label)

20609it [02:24, 143.08it/s]


In [83]:
print(X_train[:20])
print(y_train[:20])

[['숙박', '예약', '했는데', '분', '정도', '일찍', '갔더니', '대', '실비', '내고', '짐가져다놓으라', '길', '래', '그냥', '할', '먼저', '하고서', '오겟다', '한', '다음', '시', '넘어서', '왔더니', '청소', '덜', '됐다고', '기다리라는', '누가', '봐도', '제', '쓸', '방', '대실줘서', '그런거', '아닙니까', '게다가', '이불', '피', '묻어있더군요', '청소', '하신', '건지', '이불', '원래', '교체', '하는', '건지', '제', '예민한건', '가요', '방', '따', '듯', '온수', '도', '잘나오고', '나쁘지', '않았는데', '이불', '묻은', '피', '본', '순간', '진짜', '아니라는', '생각', '들었어요', '답글', '보고나니', '다신', '이용', '할', '생각', '이구', '여', '위생', '상태', '말', '한', '겁니다', '피', '묻은', '이불', '답글', '제', '시', '간문제', '일부러', '피', '묻혔다', '는걸', '해석', '되네요', '대단합니다', '밤', '열두', '시', '애', '피', '묻은걸', '발견', '햇', '늦은', '시간', '애', '전화하는게', '민폐', '일까봐', '냅둿어', '요', '후기', '써서', '기분', '나쁘', '셧', '보내요', '오라', '해도', '부탁', '해도안가여'], ['자주', '가는', '숙소', '이번', '에는', '청소', '별로', '였어요'], ['금', '연방', '인데', '담배', '냄새', '인지', '비슷한', '냄새', '계속', '나서', '숨쉬기', '불편했어요', '그래도', '잘', '쉬었다가가요'], ['대실', '시간', '길어서', '너무', '좋아요'], ['좋아요', '좋아요', '좋아요', '좋아요'], ['주차공간', '도', '없고', '방도', '먼지', '쌓여서', '더

In [84]:
X_test = []
y_test = []

for sentence, label in tqdm(zip(test_data['review_content'], test_data['label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_test.append(filtered)
        y_test.append(label)

1987it [00:13, 149.33it/s]


In [85]:
y_train = np.array(train_data['label'], dtype=int)
y_test = np.array(test_data['label'], dtype=int)

In [86]:
import pickle
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f)
with open('y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)
with open('y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [87]:
print(X_train[:10])
print(X_test[:10])
print(y_train[:10])
print(y_test[:10])

[['숙박', '예약', '했는데', '분', '정도', '일찍', '갔더니', '대', '실비', '내고', '짐가져다놓으라', '길', '래', '그냥', '할', '먼저', '하고서', '오겟다', '한', '다음', '시', '넘어서', '왔더니', '청소', '덜', '됐다고', '기다리라는', '누가', '봐도', '제', '쓸', '방', '대실줘서', '그런거', '아닙니까', '게다가', '이불', '피', '묻어있더군요', '청소', '하신', '건지', '이불', '원래', '교체', '하는', '건지', '제', '예민한건', '가요', '방', '따', '듯', '온수', '도', '잘나오고', '나쁘지', '않았는데', '이불', '묻은', '피', '본', '순간', '진짜', '아니라는', '생각', '들었어요', '답글', '보고나니', '다신', '이용', '할', '생각', '이구', '여', '위생', '상태', '말', '한', '겁니다', '피', '묻은', '이불', '답글', '제', '시', '간문제', '일부러', '피', '묻혔다', '는걸', '해석', '되네요', '대단합니다', '밤', '열두', '시', '애', '피', '묻은걸', '발견', '햇', '늦은', '시간', '애', '전화하는게', '민폐', '일까봐', '냅둿어', '요', '후기', '써서', '기분', '나쁘', '셧', '보내요', '오라', '해도', '부탁', '해도안가여'], ['자주', '가는', '숙소', '이번', '에는', '청소', '별로', '였어요'], ['금', '연방', '인데', '담배', '냄새', '인지', '비슷한', '냄새', '계속', '나서', '숨쉬기', '불편했어요', '그래도', '잘', '쉬었다가가요'], ['대실', '시간', '길어서', '너무', '좋아요'], ['좋아요', '좋아요', '좋아요', '좋아요'], ['주차공간', '도', '없고', '방도', '먼지', '쌓여서', '더